In [1]:
import tensorflow as tf
import tensorflow_text as tf_txt
import io, datetime, tqdm, re
import sentencepiece as sp
import numpy as np

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
with open("bible.txt", "r") as f:
    text = f.read()
    text = text.lower()
    text = re.sub("['\-!:%$\"]", "", text)
    text = re.sub("[0-9]", "", text)
    text = re.sub(r' +', ' ', text)
    text = re.sub(r'\n ', '\n', text)
    text = text[:100000] #smaller text for faster training
    print(text[:182])

with open("bible2.txt", "w") as f:
    f.write(text)

sp.SentencePieceTrainer.train(
    input='bible2.txt', model_prefix='tokenizer_model', model_type="unigram", vocab_size=1500
    )

the first book of moses called genesis


in the beginning god created the heaven and the earth.

and the earth was without form, and void; and darkness was upon
the face of the deep.


In [4]:
trained_tokenizer_model = tf.io.gfile.GFile('tokenizer_model.model', "rb").read()

tokenizer = tf_txt.SentencepieceTokenizer(
    model=trained_tokenizer_model, out_type=tf.int32, nbest_size=-1, alpha=1, reverse=False,
    add_bos=False, add_eos=False, return_nbest=False, name=None
)

In [5]:
tokens = tokenizer.tokenize(text)
print(tokens[:100])

tf.Tensor(
[   5  238  186  959    6  857  898   77 1338   18  910   11    5  774
   30  287    5  141    3    5   38    7    3    5   38   21  497   27
  402    4    3 1335   53    9    3  550   21   48    5  143    6    5
  552    7    3    5  219  728  365    6   30  277  523   48    5  143
    6    5  128    7    3   30   16    4   87   65   23  251    3   65
   21  251    7    3   30  149    5  251    4   15   24   21  146    3
   30  388    5  251   43    5  550    7    3   30   77    5  251   85
    4    3], shape=(100,), dtype=int32)


In [6]:
def preprocessing(data, length):
    data = tf_txt.sliding_window(data, width=length+1)
    data = tf.data.Dataset.from_tensor_slices(data)
    data = data.map(lambda x: (x[:-1], x[1:])) # create input and target
    data = data.shuffle(1000)
    data = data.batch(32)
    data = data.prefetch(20)
    return data

dataset = preprocessing(tokens, 100)

In [7]:
class Embedding(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim, length):
        super(Embedding, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.position = tf.keras.layers.Embedding(length, embedding_dim)

    def call(self, x):
        length = tf.shape(x)[-1]
        positions = tf.range(0, length)
        positions = self.position(positions)
        x = self.embedding(x)
        return x + positions

In [8]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)
        self.dense_relu = tf.keras.layers.Dense(256, activation="relu")
        self.dense = tf.keras.layers.Dense(embedding_dim, activation="relu")
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attention_output = self.attention(inputs, inputs, use_causal_mask=True)
        attention_output = self.dropout1(attention_output, training=training)
        out1 = self.layernorm1(inputs + attention_output)
        out2 = self.dense_relu(out1)
        out2 = self.dense(out2)
        out2 = self.dropout2(out2, training=training)
        return self.layernorm2(out1 + out2)

In [9]:
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, embedding_dim, num_heads, vocab_size, rate=0.1):
        super(Transformer, self).__init__()
        self.embed_dim = embedding_dim
        self.num_layers = num_layers
        self.embedding = Embedding(vocab_size, embedding_dim, 100)
        self.transformer_blocks = [TransformerBlock(embedding_dim, num_heads, rate) for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(rate)
        self.layernorm = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, training):
        x = self.embedding(inputs)
        for i in range(self.num_layers):
            x = self.transformer_blocks[i](x, training)
        x = self.dropout(x, training=training)
        x = self.layernorm(x)
        return self.dense(x)
    
    def generate_text(self, text, length, top_k):
        tokens = tokenizer.tokenize(text)
        for _ in range(length):
            logits = tf.expand_dims(tokens, 0)
            logits = self(logits, training=False)
            logits = tf.math.top_k(logits, k=top_k, sorted=True)
            sample = tf.random.uniform(shape=(), minval=0, maxval=top_k, dtype=tf.int32)
            new_word = logits.indices.numpy()[0, -1, sample]
            tokens = tf.concat([tokens, [new_word]], axis=-1)
        return tokenizer.detokenize(tokens)

In [10]:
def training_loop(model, dataset, epochs, loss_fn, optimizer):
    for epoch in range(epochs):
        for data in tqdm.tqdm(dataset):
            with tf.GradientTape() as tape:
                predictions = model(data[0], True)
                loss = loss_fn(data[1], predictions)
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            loss = loss.numpy().mean()
        print(f"Epoch: {epoch+1}; Loss: {loss}")
        tf.print(model.generate_text("and god", 20, 15))

In [11]:
model = Transformer(num_layers=4, embedding_dim=100, num_heads=4, vocab_size=1500)
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
adam = tf.keras.optimizers.Adam(learning_rate=0.001)
training_loop(model=model, dataset=dataset, epochs=10, loss_fn=loss_function, optimizer=adam)

100%|██████████| 758/758 [01:56<00:00,  6.50it/s]


Epoch: 1; Loss: 0.26514166593551636
and god was set; saying unto be my father thou of my son esau hear so. he, he the


100%|██████████| 758/758 [01:57<00:00,  6.42it/s]


Epoch: 2; Loss: 0.24761104583740234
and god will eat? bless him; because i will also and i be strohold unto my brother and after


100%|██████████| 758/758 [01:59<00:00,  6.35it/s]


...

Epoch: 8; Loss: 0.09392839670181274
and god will be blessed to abraham thee his son only thous sake nowstained his men take she said


100%|██████████| 758/758 [01:55<00:00,  6.54it/s]


Epoch: 9; Loss: 0.07282131910324097
and god of jacob sod esau upon abraham him; because isaac dwelt then digged before thou perfect. my curse


100%|██████████| 758/758 [01:49<00:00,  6.90it/s]


Epoch: 10; Loss: 0.097612164914608
and god to witiver these things i of that day; his life because she was called flyso venison


In [12]:
model.generate_text("god created", length=90, top_k=20)

<tf.Tensor: shape=(), dtype=string, numpy=b'god created sure sleep.. neither am this him forth bread and twentyion is from havilah in her and havilah when the angeleenservants me and hool i will i shall see withheldahamed that iting and k evil do by his brethren were covered the nakedness of bethel to hagar his house at beershebast the cityth year had pleaslled unto isaac they exceedingly down heteen unt jacob, but fat enar twelvey philistines looked took of'>